<a href="https://colab.research.google.com/github/dongddaoist/APPRET/blob/main/GraphVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!conda install -c conda-forge rdkit
!pip install torch torch-geometric

# Install rdkit (should only be installed via conda)

!pip install --quiet rdkit-pypi > /dev/null
!pip install optuna
# Inshttps://drive.google.com/drive/search?q=owner%3Ame%20(type%3Aapplication%2Fvnd.google.colaboratory%20%7C%7C%20type%3Aapplication%2Fvnd.google.colab)&authuser=0tall UMAP/hdbscan (recommend to do this before miniconda)
!pip install --quiet umap-learn hdbscan

!pip install pandas numpy scikit-learn


/bin/bash: line 1: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_ncc

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
from torch_geometric.utils import from_smiles, to_smiles

# Function to convert SMILES to graph data
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([atom.GetAtomicNum()])
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    return data

ifile = '/content/drive/My Drive/machine-learning/Data/fluoroether.csv'

# Load your CSV file
df = pd.read_csv(ifile)
smiles_list = df["SMILES"]

graph_data = [smiles_to_graph(smiles) for smiles in smiles_list]
graph_data = [data for data in graph_data if data is not None]

class GraphVAE(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim, latent_dim):
        super(GraphVAE, self).__init__()
        self.encoder_conv1 = GCNConv(node_feature_dim, hidden_dim)
        self.encoder_conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim * 2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim * 2, latent_dim)
        self.decoder_fc1 = nn.Linear(latent_dim, hidden_dim)
        self.decoder_conv1 = GCNConv(hidden_dim, hidden_dim)
        self.decoder_conv2 = GCNConv(hidden_dim, node_feature_dim)

    def encode(self, x, edge_index):
        h = self.encoder_conv1(x, edge_index)
        h = torch.relu(h)
        h = self.encoder_conv2(h, edge_index)
        h_mean = global_mean_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_max = global_max_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_combined = torch.cat([h_mean, h_max], dim=1)
        return self.fc_mu(h_combined), self.fc_logvar(h_combined)

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar * 0.5)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, num_nodes):
        h = self.decoder_fc1(z)
        h = torch.relu(h)
        h = h.unsqueeze(1).repeat(1, num_nodes, 1).view(-1, h.size(-1))
        return h

    def forward(self, x, edge_index):
        mu, logvar = self.encode(x, edge_index)
        z = self.reparameterize(mu, logvar)
        num_nodes = x.size(0)
        return self.decode(z, num_nodes), mu, logvar

def train_graphvae(model, data_loader, epochs=100, learning_rate=1e-2):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for data in data_loader:
            data = data.to(device)
            optimizer.zero_grad()
            x_recon, mu, logvar = model(data.x, data.edge_index)

            recon_loss = criterion(x_recon, data.x)
            kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + kl_divergence
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(data_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Example usage
node_feature_dim = 1  # Single feature: atomic number
hidden_dim = 128
latent_dim = 64

graphvae = GraphVAE(node_feature_dim, hidden_dim, latent_dim).to(device)
data_loader = DataLoader(graph_data, batch_size=32, shuffle=True)

train_graphvae(graphvae, data_loader)


# Hyper-parameter Optimization

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_parallel_coordinate
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
from torch_geometric.utils import from_smiles, to_smiles

# Function to convert SMILES to graph data
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([atom.GetAtomicNum()])
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    return data

ifile = '/content/drive/My Drive/machine-learning/Data/fluoroether.csv'

# Load your CSV file
df = pd.read_csv(ifile)
smiles_list = df["SMILES"]

graph_data = [smiles_to_graph(smiles) for smiles in smiles_list]
graph_data = [data for data in graph_data if data is not None]

class GraphVAE(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim, latent_dim, num_hidden_layers):
        super(GraphVAE, self).__init__()
        self.encoder_convs = nn.ModuleList([GCNConv(node_feature_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_hidden_layers)])
        self.fc_mu = nn.Linear(hidden_dim * 2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim * 2, latent_dim)
        self.decoder_fc1 = nn.Linear(latent_dim, hidden_dim)
        self.decoder_conv1 = GCNConv(hidden_dim, hidden_dim)
        self.decoder_conv2 = GCNConv(hidden_dim, node_feature_dim)

    def encode(self, x, edge_index):
        h = x
        for conv in self.encoder_convs:
            h = conv(h, edge_index)
            h = torch.relu(h)
        h_mean = global_mean_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_max = global_max_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_combined = torch.cat([h_mean, h_max], dim=1)
        return self.fc_mu(h_combined), self.fc_logvar(h_combined)

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar * 0.5)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, num_nodes):
        h = self.decoder_fc1(z)
        h = torch.relu(h)
        h = h.unsqueeze(1).repeat(1, num_nodes, 1).view(-1, h.size(-1))
        return h

    def forward(self, x, edge_index):
        mu, logvar = self.encode(x, edge_index)
        z = self.reparameterize(mu, logvar)
        num_nodes = x.size(0)
        return self.decode(z, num_nodes), mu, logvar

def train_graphvae(model, data_loader, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0.0
        for data in data_loader:
            data = data.to(device)
            optimizer.zero_grad()
            x_recon, mu, logvar = model(data.x, data.edge_index)

            recon_loss = criterion(x_recon, data.x)
            kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + kl_divergence
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(data_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

    return avg_loss

def objective(trial):
    # Define the hyperparameters to be optimized
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 3e-2)
    hidden_dim = trial.suggest_categorical('hidden_dim', [64, 128, 192, 256])
    latent_dim = trial.suggest_categorical('latent_dim', [32, 64, 128])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = 100 #trial.suggest_int('epochs', 50, 100)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 2, 6)

    # Create the model and data loader
    graphvae = GraphVAE(node_feature_dim=1, hidden_dim=hidden_dim, latent_dim=latent_dim, num_hidden_layers=num_hidden_layers).to(device)
    data_loader = DataLoader(graph_data, batch_size=batch_size, shuffle=True)

    # Train the model
    avg_loss = train_graphvae(graphvae, data_loader, epochs=epochs, learning_rate=learning_rate)

    return avg_loss

def save_intermediate_results(study, trial):
    with open('/content/drive/My Drive/machine-learning/generative/GraphVAE/optimization.txt', 'a') as f:
        f.write(f'Trial {trial.number}: Loss={trial.value}, Params={trial.params}\n')

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, callbacks=[save_intermediate_results])

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

# Save the best hyperparameters to a file
with open('/content/drive/My Drive/machine-learning/generative/GraphVAE/optimization.txt', 'a') as f:
    f.write(f'Best hyperparameters: {study.best_params}\n')
    f.write(f'Best loss: {study.best_value}\n')


# Plot optimization history
optuna.visualization.plot_optimization_history(study).show()

# Plot hyperparameter importance
optuna.visualization.plot_param_importances(study).show()

# Plot parallel coordinate plot
optuna.visualization.plot_parallel_coordinate(study).show()


[I 2024-08-09 13:49:35,194] A new study created in memory with name: no-name-01a96f19-eaad-49b0-b0fa-ff3f0bc6e25d
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning:

'data.DataLoader' is deprecated, use 'loader.DataLoader' instead

/usr/local/lib/python3.10/dist-packages/torch_geometric/warnings.py:11: UserWarning:

The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([443, 1])) that is different to the input size (torch.Size([443, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([34, 1])) that is different to the input size (torch.Size([34, 128])). This will likely lead to incorrect results due to broa

Epoch 100, Loss: 11.024536609649658


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([456, 1])) that is different to the input size (torch.Size([456, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([21, 1])) that is different to the input size (torch.Size([21, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([453, 1])) that is different to the input size (torch.Size([453, 256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([24, 1]

Epoch 100, Loss: 9.784942150115967


[I 2024-08-09 13:49:39,357] Trial 2 finished with value: 55.31134033203125 and parameters: {'learning_rate': 0.002446855819168861, 'hidden_dim': 256, 'latent_dim': 32, 'batch_size': 64, 'num_hidden_layers': 5}. Best is trial 1 with value: 9.784942150115967.


...

Epoch 100, Loss: 8.206456820170084


[I 2024-08-09 13:52:39,562] Trial 98 finished with value: 7.004858493804932 and parameters: {'learning_rate': 0.01743791677780444, 'hidden_dim': 64, 'latent_dim': 128, 'batch_size': 16, 'num_hidden_layers': 5}. Best is trial 96 with value: 5.63440736134847.


Epoch 100, Loss: 7.004858493804932


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([186, 1])) that is different to the input size (torch.Size([186, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

[I 2024-08-09 13:52:42,062] Trial 99 finished with value: 19.068155606587727 and parameters: {'learning_rate': 0.01911554162135102, 'hidden_dim': 64, 'latent_dim': 128, 'batch_size': 16, 'num_hidden_layers': 5}. Best is trial 96 with value: 5.63440736134847.


Epoch 100, Loss: 19.068155606587727
Best hyperparameters:  {'learning_rate': 0.007831007398929793, 'hidden_dim': 64, 'latent_dim': 128, 'batch_size': 16, 'num_hidden_layers': 5}


# Use optimized parameters

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_parallel_coordinate
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
from torch_geometric.utils import from_smiles, to_smiles

# Function to convert SMILES to graph data
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([atom.GetAtomicNum()])
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    return data

ifile = '/content/drive/My Drive/machine-learning/Data/fluoroether.csv'

# Load your CSV file
df = pd.read_csv(ifile)
smiles_list = df["SMILES"]

graph_data = [smiles_to_graph(smiles) for smiles in smiles_list]
graph_data = [data for data in graph_data if data is not None]

class GraphVAE(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim, latent_dim, num_hidden_layers):
        super(GraphVAE, self).__init__()
        self.encoder_convs = nn.ModuleList([GCNConv(node_feature_dim if i == 0 else hidden_dim, hidden_dim) for i in range(num_hidden_layers)])
        self.fc_mu = nn.Linear(hidden_dim * 2, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim * 2, latent_dim)
        self.decoder_fc1 = nn.Linear(latent_dim, hidden_dim)
        self.decoder_conv1 = GCNConv(hidden_dim, hidden_dim)
        self.decoder_conv2 = GCNConv(hidden_dim, node_feature_dim)

    def encode(self, x, edge_index):
        h = x
        for conv in self.encoder_convs:
            h = conv(h, edge_index)
            h = torch.relu(h)
        h_mean = global_mean_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_max = global_max_pool(h, torch.zeros(h.size(0), dtype=torch.long).to(h.device))
        h_combined = torch.cat([h_mean, h_max], dim=1)
        return self.fc_mu(h_combined), self.fc_logvar(h_combined)

    def reparameterize(self, mu, logvar):
        std = torch.exp(logvar * 0.5)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, num_nodes):
        h = self.decoder_fc1(z)
        h = torch.relu(h)
        h = h.unsqueeze(1).repeat(1, num_nodes, 1).view(-1, h.size(-1))
        return h

    def forward(self, x, edge_index):
        mu, logvar = self.encode(x, edge_index)
        z = self.reparameterize(mu, logvar)
        num_nodes = x.size(0)
        return self.decode(z, num_nodes), mu, logvar

def train_graphvae(model, data_loader, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    model.train()

    for epoch in range(epochs):
        epoch_loss = 0.0
        for data in data_loader:
            data = data.to(device)
            optimizer.zero_grad()
            x_recon, mu, logvar = model(data.x, data.edge_index)

            recon_loss = criterion(x_recon, data.x)
            kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + kl_divergence
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(data_loader)
        if ((epoch+1) % 10 == 0 ) or (epoch==0):
            print(f"Epoch {epoch + 1}, Loss: {avg_loss}")
       # print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

    return avg_loss


learning_rate =  0.008

hidden_dim = 64
latent_dim = 128
batch_size = 16
max_epochs = 2000
num_hidden_layers = 5
    # Create the model and data loader
graphvae = GraphVAE(node_feature_dim=1, hidden_dim=hidden_dim, latent_dim=latent_dim, num_hidden_layers=num_hidden_layers).to(device)
data_loader = DataLoader(graph_data, batch_size=batch_size, shuffle=True)

    # Train the model
avg_loss = train_graphvae(graphvae, data_loader, epochs=max_epochs, learning_rate=learning_rate)

# Save the trained model
torch.save(graphvae.state_dict(), '/content/drive/My Drive/machine-learning/generative/GraphVAE/best_graphvae_model.pth')

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning:

'data.DataLoader' is deprecated, use 'loader.DataLoader' instead

/usr/local/lib/python3.10/dist-packages/torch_geometric/warnings.py:11: UserWarning:

The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([212, 1])) that is different to the input size (torch.Size([212, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([219, 1])) that is different to the input size (torch.Size([219, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py

Epoch 1, Loss: 58.399725596110024


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([26, 1])) that is different to the input size (torch.Size([26, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([243, 1])) that is different to the input size (torch.Size([243, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([205, 1])) that is different to the input size (torch.Size([205, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([29, 1])) 

Epoch 10, Loss: 38.78242619832357


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([239, 1])) that is different to the input size (torch.Size([239, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([211, 1])) that is different to the input size (torch.Size([211, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([27, 1])) that is different to the input size (torch.Size([27, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([31, 1])) 

...

Epoch 940, Loss: 2.6185452938079834
Epoch 950, Loss: 4.053299347559611
Epoch 960, Loss: 2.831301291783651
Epoch 970, Loss: 3.7148247559865317
Epoch 980, Loss: 3.1352707544962564
Epoch 990, Loss: 2.829050143559774
Epoch 1000, Loss: 3.081733226776123
Epoch 1010, Loss: 2.253883202870687
Epoch 1020, Loss: 2.2308369477589927
Epoch 1030, Loss: 3.287049929300944
Epoch 1040, Loss: 2.887331247329712
Epoch 1050, Loss: 2.6878906885782876
Epoch 1060, Loss: 2.748896519343058


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([183, 1])) that is different to the input size (torch.Size([183, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1070, Loss: 2.4814990361531577
Epoch 1080, Loss: 2.1563851038614907
Epoch 1090, Loss: 2.5296533902486167
Epoch 1100, Loss: 2.282858371734619
Epoch 1110, Loss: 2.1524195671081543
Epoch 1120, Loss: 2.490468184153239
Epoch 1130, Loss: 2.222184975941976
Epoch 1140, Loss: 2.1606273651123047
Epoch 1150, Loss: 2.2985075314839682
Epoch 1160, Loss: 2.060397227605184
Epoch 1170, Loss: 2.1418632666269937
Epoch 1180, Loss: 2.114960193634033
Epoch 1190, Loss: 2.059791167577108
Epoch 1200, Loss: 2.074927568435669
Epoch 1210, Loss: 2.0255820751190186
Epoch 1220, Loss: 1.9968716700871785
Epoch 1230, Loss: 2.071328639984131
Epoch 1240, Loss: 2.008789340655009
Epoch 1250, Loss: 1.9942965507507324
Epoch 1260, Loss: 2.01377534866333
Epoch 1270, Loss: 2.0529563426971436
Epoch 1280, Loss: 2.040154973665873
Epoch 1290, Loss: 2.044823169708252
Epoch 1300, Loss: 2.0231420596440635
Epoch 1310, Loss: 2.035458962122599
Epoch 1320, Loss: 2.0459084510803223
Epoch 1330, Loss: 2.0346455176671348
Epoch 1340, Los

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([175, 1])) that is different to the input size (torch.Size([175, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([172, 1])) that is different to the input size (torch.Size([172, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning:

Using a target size (torch.Size([271, 1])) that is different to the input size (torch.Size([271, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Epoch 1510, Loss: 2.0870211124420166
Epoch 1520, Loss: 2.014110525449117
Epoch 1530, Loss: 2.0216628313064575
Epoch 1540, Loss: 2.0259200731913247
Epoch 1550, Loss: 2.048696994781494
Epoch 1560, Loss: 2.1818774143854776
Epoch 1570, Loss: 1.9896570841471355
Epoch 1580, Loss: 2.136410633722941
Epoch 1590, Loss: 2.233964681625366
Epoch 1600, Loss: 2.078770637512207
Epoch 1610, Loss: 2.0411442120869956
Epoch 1620, Loss: 2.0878215630849204
Epoch 1630, Loss: 2.0446290969848633
Epoch 1640, Loss: 2.339623212814331
Epoch 1650, Loss: 2.158522129058838
Epoch 1660, Loss: 2.111853917439779
Epoch 1670, Loss: 2.165721575419108
Epoch 1680, Loss: 2.03033447265625
Epoch 1690, Loss: 2.06772788365682
Epoch 1700, Loss: 2.103665590286255
Epoch 1710, Loss: 1.9725677172342937
Epoch 1720, Loss: 2.063175360361735
Epoch 1730, Loss: 2.433592120806376
Epoch 1740, Loss: 2.1427489121754966
Epoch 1750, Loss: 2.045745611190796
Epoch 1760, Loss: 2.543100039164225
Epoch 1770, Loss: 2.4124279022216797
Epoch 1780, Loss: 2

In [ ]:
graphvae


GraphVAE(
  (encoder_convs): ModuleList(
    (0): GCNConv(1, 64)
    (1-4): 4 x GCNConv(64, 64)
  )
  (fc_mu): Linear(in_features=128, out_features=128, bias=True)
  (fc_logvar): Linear(in_features=128, out_features=128, bias=True)
  (decoder_fc1): Linear(in_features=128, out_features=64, bias=True)
  (decoder_conv1): GCNConv(64, 64)
  (decoder_conv2): GCNConv(64, 1)
)

In [ ]:
# Load the trained model
def load_model(file_path, node_feature_dim, hidden_dim, latent_dim, num_hidden_layers):
    model = GraphVAE(node_feature_dim=node_feature_dim, hidden_dim=hidden_dim, latent_dim=latent_dim, num_hidden_layers=num_hidden_layers)
    model.load_state_dict(torch.load(file_path))
    model.eval()
    return model

# Function to generate new molecules
def generate_molecules(model, num_samples=10):
    model.eval()
    generated_smiles = []

    with torch.no_grad():
        for _ in range(num_samples):
            z = torch.randn(1, model.fc_mu.out_features).to(device)
            # Assuming a fixed number of nodes for simplicity
            num_nodes = 30  # Adjust based on your data
            edge_index = torch.tensor([[i, i+1] for i in range(num_nodes-1)] + [[i+1, i] for i in range(num_nodes-1)], dtype=torch.long).t().contiguous().to(device)
            h = model.decode(z, num_nodes)
            smiles = to_smiles(Data(x=h, edge_index=edge_index))
            if Chem.MolFromSmiles(smiles):  # Validate SMILES
                generated_smiles.append(smiles)

    return generated_smiles

# Load the trained model
trained_model = load_model('best_graphvae_model.pth', node_feature_dim=1, hidden_dim=best_params['hidden_dim'], latent_dim=best_params['latent_dim'], num_hidden_layers=best_params['num_hidden_layers']).to(device)

# Generate new molecules
new_smiles = generate_molecules(trained_model)
print("Generated SMILES:")
for smiles in new_smiles:
    print(smiles)

# Write the generated SMILES to a new file
output_file = 'generated_smiles.txt'
with open(output_file, 'w') as f:
    for smiles in new_smiles:
        f.write(smiles + '\n')


Generate new molecules